# Challenge 1: Multi-Channel Contacts

In [1]:
import json
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import LabelEncoder

## Load Data

In [2]:
# Load data
df = pd.read_json('contacts.json')
df.loc[df.Email == '', 'Email'] = 'Missing'
df.loc[df.Phone == '', 'Phone'] = 'Missing'
df.loc[df.OrderId == '', 'OrderId'] = 'Missing'
df.shape

(500000, 5)

In [3]:
# Create edges
graph_df = pd.concat([
    pd.DataFrame({'source': df.Email, 'target': df.Phone}),
    pd.DataFrame({'source': df.Email, 'target': df.OrderId}),
    pd.DataFrame({'source': df.Phone, 'target': df.OrderId}),
])

graph_properties = pd.concat([
    pd.DataFrame({'node': df.Email.unique()}),
    pd.DataFrame({'node': df.Phone.unique()}),
    pd.DataFrame({'node': df.OrderId.unique()}),
]).reset_index(drop=True)

# graph_properties = graph_properties[graph_properties.node != 'Missing']

# Create node IDs
le = LabelEncoder()
graph_properties['node_id'] = le.fit_transform(graph_properties.node)

# Convert nodes in graph_df to their node IDs
graph_df['source'] = le.transform(graph_df[['source']])
graph_df['target'] = le.transform(graph_df[['target']])

# Attach node IDs to main table
df['email_id'] = le.transform(df[['Email']])
df['phone_id'] = le.transform(df[['Phone']])
df['orderid_id'] = le.transform(df[['OrderId']])

missing_id = le.transform(np.array(['Missing']))[0]
print(f"Missing label: {missing_id}")

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Missing label: 297623


In [4]:
# Set up graph
g1 = nx.from_pandas_edgelist(graph_df, source='source', target='target')
g1.remove_node(missing_id)

# Get connected subgraphs
groups = list(nx.connected_components(g1))

In [5]:
# Create group lookup table
group_lookup = pd.DataFrame(groups).apply(lambda x: [int(i) for i in x if not np.isnan(i)], axis=1).reset_index().explode(0).rename(columns={'index': 'group', 0: 'tic_id'})

In [6]:
df.head()

,Id,Email,Phone,Contacts,OrderId,email_id,phone_id,orderid_id
0,0,gkzAbIy@qq.com,Missing,1,Missing,466740,297623,297623
1,1,Missing,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,297623,63289,587459
2,2,Missing,9125983679,0,Missing,297623,174086,297623
3,3,mdllpYmE@gmail.com,Missing,0,bHquEnCbbsGLqllwryxPsNOxa,516035,297623,419954
4,4,Missing,300364407,2,Missing,297623,57779,297623


In [14]:
# Output
output = df.copy()

# Get IDs
output['email_match_id'] = pd.merge(df, group_lookup, how='left', left_on='email_id', right_on='tic_id')['group']
output['phone_match_id'] = pd.merge(output, group_lookup, how='left', left_on='phone_id', right_on='tic_id')['group']
output['orderid_match_id'] = pd.merge(output, group_lookup, how='left', left_on='orderid_id', right_on='tic_id')['group']

In [15]:
output.head(100)

,Id,Email,Phone,Contacts,OrderId,email_id,phone_id,orderid_id,email_match_id,phone_match_id,orderid_match_id
0,0,gkzAbIy@qq.com,Missing,1,Missing,466740,297623,297623,0.0,NaN,NaN
1,1,Missing,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,297623,63289,587459,NaN,1.0,1.0
2,2,Missing,9125983679,0,Missing,297623,174086,297623,NaN,2.0,NaN
3,3,mdllpYmE@gmail.com,Missing,0,bHquEnCbbsGLqllwryxPsNOxa,516035,297623,419954,3.0,NaN,3.0
4,4,Missing,300364407,2,Missing,297623,57779,297623,NaN,4.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
95,95,gWNVijaZJrQUSBhxHi@qq.com,Missing,0,Missing,464271,297623,297623,90.0,NaN,NaN
96,96,Missing,468981077,4,CVGixrrHHbvLmNAhamcnUQHrR,297623,89747,211115,NaN,91.0,91.0
97,97,FoDkCwaAyfreFvCqPcBC@yahoo.com,Missing,3,Missing,239464,297623,297623,92.0,NaN,NaN
98,98,DxtVenmxYlNSRxG@qq.com,Missing,0,Missing,224273,297623,297623,93.0,NaN,NaN


In [16]:
# Extract group ID
output['group_id'] = output[['email_match_id', 'phone_match_id', 'orderid_match_id']].mean(axis=1)

In [17]:
# Prepare answer
output['ticket_trace'] = output.groupby('group_id')['Id'].transform(lambda x: '-'.join([str(y) for y in sorted(x)]))
output['contact'] = output.groupby('group_id')['Contacts'].transform(sum)
output['ticket_trace/contact'] = output.ticket_trace + ', ' + output.contact.astype(str)
output = output.rename(columns={'Id': 'ticket_id'})

In [18]:
output.head()

,ticket_id,Email,Phone,Contacts,OrderId,email_id,phone_id,orderid_id,email_match_id,phone_match_id,orderid_match_id,group_id,ticket_trace,contact,ticket_trace/contact
0,0,gkzAbIy@qq.com,Missing,1,Missing,466740,297623,297623,0.0,NaN,NaN,0.0,0,1,"0, 1"
1,1,Missing,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,297623,63289,587459,NaN,1.0,1.0,1.0,1-2458-98519-115061-140081-165605-476346,12,"1-2458-98519-115061-140081-165605-476346, 12"
2,2,Missing,9125983679,0,Missing,297623,174086,297623,NaN,2.0,NaN,2.0,2-159312-322639-348955,4,"2-159312-322639-348955, 4"
3,3,mdllpYmE@gmail.com,Missing,0,bHquEnCbbsGLqllwryxPsNOxa,516035,297623,419954,3.0,NaN,3.0,3.0,3,0,"3, 0"
4,4,Missing,300364407,2,Missing,297623,57779,297623,NaN,4.0,NaN,4.0,4,2,"4, 2"


In [19]:
output[['ticket_id', 'ticket_trace/contact']].to_csv('#1 - Datards - Open.csv', index=False)